In [1]:
import numpy as np
import os
import sys

import tensorflow as tf
from tensorflow.keras.layers import *

import hls4ml
from hls4ml.model.profiling import numerical, activations_keras, boxplot

sys.path.append('/usersc/bz18310/previous_notebook/cms-l1-triggers')

from utils.analysis import eff_rate, optimal_eff_rate
from utils.preprocessing import resize
from utils.plotting import *
from utils.hls4ml_helpers import *

In [35]:
def make_model(in_shape):

    model = tf.keras.Sequential()

    model.add(Conv2D(4, kernel_size=(3,3), input_shape=in_shape))
    model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))

    model.add(Conv2D(8, kernel_size=(3,3)))
    model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(24))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('sigmoid'))

    model.build(input_shape=in_shape)

    model.summary()
    
    model = compiler(model)
    
    return model

def compiler(model_name):
    opt = tf.keras.optimizers.Adam(0.001)
    sensitivity_metric = tf.keras.metrics.SensitivityAtSpecificity(name='sens_at_spec',
                                                                             specificity=0.99925,     
                                                                             num_thresholds=20000)     
    auc_metric = tf.keras.metrics.AUC(name='auc', num_thresholds=200)   
    metrics = ['accuracy', sensitivity_metric, auc_metric]

    model_name.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)
    
    return model_name

def hls4ml_converter(model_name, directory):
        
    config = hls4ml.utils.config_from_keras_model(model_name, granularity='model')
    
    #"""
    config = {'Model': {'Precision': 'ap_fixed<5,1>',
              'ReuseFactor': 1,
              'Strategy': 'Latency'}}
    #"""
    print("-----------------------------------")
    print("Configuration")
    display(config)
    print("-----------------------------------")
    hls_model = hls4ml.converters.convert_from_keras_model(model_name,
                                                           hls_config=config,   #set configuration of model
                                                           output_dir=directory,  #6bit model save
                                                           fpga_part='xcku15p-ffva1760-2-e',     #What?
                                                            clock_period=(1/.24),      #set clock period??
                                                              io_type='io_stream')     #
    hls_model.compile()
    return hls_model

In [39]:
#input_shapes = ((20,12,1), (40,24,1), (60,36,1), (80,48,1), (100,60,1), (120,72,1))
input_shapes = ((25,15,1), (30,18,1), (35,21,1), (40, 24, 1))

a = './input_shapes/'

directories = []

def remove_spaces(string):
    a = string.replace(",", "_")
    return a.replace(" ", "")

for i in range(len(input_shapes)):
    b = str(input_shapes[i])
    b = remove_spaces(b)
    directories.append(a + b + '/hls4ml_prj')

In [37]:
for i in range(len(input_shapes)):
    model = make_model(input_shapes[i])
    hls4ml_model = hls4ml_converter(model, directories[i])
    hls4ml_model.build(csim=False)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 23, 13, 4)         40        
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 11, 6, 4)          0         
_________________________________________________________________
batch_normalization_72 (Batc (None, 11, 6, 4)          44        
_________________________________________________________________
activation_72 (Activation)   (None, 11, 6, 4)          0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 9, 4, 8)           296       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 4, 2, 8)           0         
_________________________________________________________________
batch_normalization_73 (Batc (None, 4, 2, 8)         

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_36_input, layer type: InputLayer, current shape: [[None, 25, 15, 1]]
Layer name: conv2d_36, layer type: Conv2D, current shape: [[None, 25, 15, 1]]
Layer name: max_pooling2d_36, layer type: MaxPooling2D, current shape: [[None, 23, 13, 4]]
Layer name: batch_normalization_72, layer type: BatchNormalization, current shape: [[None, 11, 6, 4]]
Layer name: activation_72, layer type: Activation, current shape: [[None, 11, 6, 4]]
Layer name: conv2d_37, layer type: Conv2D, current shape: [[None, 11, 6, 4]]
Layer name: max_pooling2d_37, layer type: MaxPooling2D, current shape: [[None, 9, 4, 8]]
Layer name: batch_normalization_73, layer type: BatchNormalization, current shape: [[None, 4, 2, 8]]
Layer name: activation_73, layer type: Activation, current shape: [[None, 4, 2, 8]]
Layer name: dense_36, layer type: Dense, current shape: [[None, 4, 2, 8]]
Layer name: batch_normalization_74, layer type: BatchNormaliz

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_38_input, layer type: InputLayer, current shape: [[None, 30, 18, 1]]
Layer name: conv2d_38, layer type: Conv2D, current shape: [[None, 30, 18, 1]]
Layer name: max_pooling2d_38, layer type: MaxPooling2D, current shape: [[None, 28, 16, 4]]
Layer name: batch_normalization_76, layer type: BatchNormalization, current shape: [[None, 14, 8, 4]]
Layer name: activation_76, layer type: Activation, current shape: [[None, 14, 8, 4]]
Layer name: conv2d_39, layer type: Conv2D, current shape: [[None, 14, 8, 4]]
Layer name: max_pooling2d_39, layer type: MaxPooling2D, current shape: [[None, 12, 6, 8]]
Layer name: batch_normalization_77, layer type: BatchNormalization, current shape: [[None, 6, 3, 8]]
Layer name: activation_77, layer type: Activation, current shape: [[None, 6, 3, 8]]
Layer name: dense_38, layer type: Dense, current shape: [[None, 6, 3, 8]]
Layer name: batch_normalization_78, layer type: BatchNormali

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_40_input, layer type: InputLayer, current shape: [[None, 35, 21, 1]]
Layer name: conv2d_40, layer type: Conv2D, current shape: [[None, 35, 21, 1]]
Layer name: max_pooling2d_40, layer type: MaxPooling2D, current shape: [[None, 33, 19, 4]]
Layer name: batch_normalization_80, layer type: BatchNormalization, current shape: [[None, 16, 9, 4]]
Layer name: activation_80, layer type: Activation, current shape: [[None, 16, 9, 4]]
Layer name: conv2d_41, layer type: Conv2D, current shape: [[None, 16, 9, 4]]
Layer name: max_pooling2d_41, layer type: MaxPooling2D, current shape: [[None, 14, 7, 8]]
Layer name: batch_normalization_81, layer type: BatchNormalization, current shape: [[None, 7, 3, 8]]
Layer name: activation_81, layer type: Activation, current shape: [[None, 7, 3, 8]]
Layer name: dense_40, layer type: Dense, current shape: [[None, 7, 3, 8]]
Layer name: batch_normalization_82, layer type: BatchNormali

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_42_input, layer type: InputLayer, current shape: [[None, 40, 24, 1]]
Layer name: conv2d_42, layer type: Conv2D, current shape: [[None, 40, 24, 1]]
Layer name: max_pooling2d_42, layer type: MaxPooling2D, current shape: [[None, 38, 22, 4]]
Layer name: batch_normalization_84, layer type: BatchNormalization, current shape: [[None, 19, 11, 4]]
Layer name: activation_84, layer type: Activation, current shape: [[None, 19, 11, 4]]
Layer name: conv2d_43, layer type: Conv2D, current shape: [[None, 19, 11, 4]]
Layer name: max_pooling2d_43, layer type: MaxPooling2D, current shape: [[None, 17, 9, 8]]
Layer name: batch_normalization_85, layer type: BatchNormalization, current shape: [[None, 8, 4, 8]]
Layer name: activation_85, layer type: Activation, current shape: [[None, 8, 4, 8]]
Layer name: dense_42, layer type: Dense, current shape: [[None, 8, 4, 8]]
Layer name: batch_normalization_86, layer type: BatchNorm

In [40]:
for d in directories:
    hls4ml.report.read_vivado_report(d)

Found 1 solution(s) in ./input_shapes/(25_15_1)/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Tue Jul 27 14:43:35 2021

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: kintexuplus
* Target device:  xcku15p-ffva1760-2-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 4.17 ns | 3.646 ns |   0.52 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---

In [24]:
model = make_model(input_shapes[1])
hls4ml_model = hls4ml_converter(model, directories[1])
hls4ml_model.build(csim=False)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 38, 22, 4)         40        
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 19, 11, 4)         0         
_________________________________________________________________
batch_normalization_56 (Batc (None, 19, 11, 4)         76        
_________________________________________________________________
activation_56 (Activation)   (None, 19, 11, 4)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 17, 9, 8)          296       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 8, 4, 8)           0         
_________________________________________________________________
batch_normalization_57 (Batc (None, 8, 4, 8)         

{'Model': {'Precision': 'ap_fixed<5,1>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'}}

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_28_input, layer type: InputLayer, current shape: [[None, 40, 24, 1]]
Layer name: conv2d_28, layer type: Conv2D, current shape: [[None, 40, 24, 1]]
Layer name: max_pooling2d_28, layer type: MaxPooling2D, current shape: [[None, 38, 22, 4]]
Layer name: batch_normalization_56, layer type: BatchNormalization, current shape: [[None, 19, 11, 4]]
Layer name: activation_56, layer type: Activation, current shape: [[None, 19, 11, 4]]
Layer name: conv2d_29, layer type: Conv2D, current shape: [[None, 19, 11, 4]]
Layer name: max_pooling2d_29, layer type: MaxPooling2D, current shape: [[None, 17, 9, 8]]
Layer name: batch_normalization_57, layer type: BatchNormalization, current shape: [[None, 8, 4, 8]]
Layer name: activation_57, layer type: Activation, current shape: [[None, 8, 4, 8]]
Layer name: dense_28, layer type: Dense, current shape: [[None, 8, 4, 8]]
Layer name: batch_normalization_58, layer type: BatchNorm

{}

In [25]:
hls4ml.report.read_vivado_report(directories[1])

Found 1 solution(s) in ./input_shapes/(40_24_1)5_bit/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
Synthesis report not found.
Co-simulation report not found.


In [41]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [45]:
hls4ml.report.read_vivado_report('./input_shapes/(20_12_1)/hls4ml_prj')

Found 1 solution(s) in ./input_shapes/(20_12_1)/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Tue Jul 27 13:54:08 2021

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: kintexuplus
* Target device:  xcku15p-ffva1760-2-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 4.17 ns | 3.646 ns |   0.52 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---

In [44]:
print(directories)


['./input_shapes/(25_15_1)/hls4ml_prj', './input_shapes/(30_18_1)/hls4ml_prj', './input_shapes/(35_21_1)/hls4ml_prj', './input_shapes/(40_24_1)/hls4ml_prj']
